# Convolutional Neural Network

## Packages

In [ ]:
# ----- Tensorflow -----
import tensorflow as tf
from keras.models import Model, clone_model
from keras.layers import Flatten, Dense, Input
from keras import optimizers as opt

# ----- Transfer learning -----
from keras.applications import NASNetLarge
from keras.applications import EfficientNetB0
from keras.applications import MobileNetV2

# ----- plot -----
import matplotlib.pyplot as plt

# ----- Utility functions -----
from utils import load_data, plot_hist, plot_hist_sideways, _convert_sample

# ------ Data ------
import numpy as np


## Load Data

In [ ]:
data_dir = "D:/OneDrive - Syddansk Universitet/kandidat/3_semester/Maskinlæring/ds807_anvendt_maskinlæring/eksamen/exam/patch_camelyon"
BATCH_SIZE = 32
train, test, val = load_data(data_dir, perc=10, batch_size=BATCH_SIZE)

## Transfer Learning
- Discuss and apply transfer learning. Motivate what type of transfer learning you use and how you apply it, including considerations for how to prepare the data for this. Here, be sure to visualize plots of train and validation losses and accuracies.



In [ ]:
# Hyperparams
EPOCHS = 100
LEARNING_RATE = 0.001
OPTIMIZER = opt.Nadam(learning_rate = LEARNING_RATE)

### Fine Tuning

#### Base Model

In [ ]:
# Initilize base model, choose the best one
base_model = MobileNetV2(
    input_shape=(96, 96, 3),
    include_top= False,
    weights='imagenet'
)
base_model.trainable = False

#### Classifier

In [ ]:
# Build classifer
# Chose the best performing
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)
full_model = Model(inputs = base_model.input, outputs = output)

# Compile model
full_model.compile(
    optimizer = OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )

### Grid Search over Fine Tuning

In [7]:
histories = []
models = [] # Just in case we want to access one of the submodels
STEP = 15
start_layer_slice = len(full_model.layers) - STEP
for i in range(len(full_model.layers) - 1, 0, - STEP):
    end_layer_slice = i
    for layer in full_model.layers[start_layer_slice : end_layer_slice]:
        layer.trainable = True
    
    # Create a clone of model
    m = clone_model(full_model)
    m.compile(
    optimizer = OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )    
    
    # Train clone
    hist = m.fit(train, validation_data = val, epochs = 10)
    histories.append(hist)
    
    # Append stats
    models.append(m)

    # Guard
    start_layer_slice -= STEP
    if start_layer_slice < 0:
        start_layer_slice = 0


820/820 [==============================] - 376s 458ms/step - loss: 0.4379 - accuracy: 0.8044 - val_loss: 2.9297 - val_accuracy: 0.4931
Epoch 7/10
820/820 [==============================] - 430s 524ms/step - loss: 0.4225 - accuracy: 0.8123 - val_loss: 3.1763 - val_accuracy: 0.4931
Epoch 8/10
820/820 [==============================] - 420s 512ms/step - loss: 0.4058 - accuracy: 0.8220 - val_loss: 2.8869 - val_accuracy: 0.4931
Epoch 9/10
820/820 [==============================] - 414s 504ms/step - loss: 0.3850 - accuracy: 0.8319 - val_loss: 2.7812 - val_accuracy: 0.4931
Epoch 10/10
820/820 [==============================] - 428s 522ms/step - loss: 0.3656 - accuracy: 0.8429 - val_loss: 1.2879 - val_accuracy: 0.4931
Epoch 1/10
820/820 [==============================] - 485s 578ms/step - loss: 0.7051 - accuracy: 0.7745 - val_loss: 0.7782 - val_accuracy: 0.5069
Epoch 2/10
820/820 [==============================] - 470s 574ms/step - loss: 0.4251 - accuracy: 0.8110 - val_loss: 0.7353 - val_accur

KeyboardInterrupt: 

### Plot results

In [ ]:
start_layer = len(full_model.layers) - 1
end_layer = len(full_model.layers)
for i, history in enumerate(histories):
    start_layer -= STEP
    if start_layer < 0:
        start_layer = 0
    
    history_dict = history.history
    # Plot 1 values
    loss_values = history_dict['loss']
    val_loss_values = history_dict['val_loss']

    # Plot 2 values
    acc_values = history_dict['accuracy']
    val_values = history_dict['val_accuracy']

    epochs = range(1, len(loss_values) + 1)

    # Plot 1
    plt.subplot(1,2,1)
    plt.plot(epochs, loss_values, 'r', label='Training loss') # 'bo' is for blue dot, 'b' is for solid blue line
    plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
    plt.title(f'Training and validation loss: \n Trainable Layers {start_layer} - {end_layer}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot 2
    plt.subplot(1,2,2)
    plt.plot(epochs, acc_values, 'r', label='Training accuracy')
    plt.plot(epochs, val_values, 'b', label='Validation accuracy')
    plt.title(f'Training and validation accuracy: \n Trainable Layers {start_layer} - {end_layer}')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.savefig(f"C:/Users/chris/Desktop/applied_ML_faelles/exam/question_2/fine_tuning_res/fine_tuning - Trainable Layers {start_layer} - {end_layer}.png")
    plt.show()


## Final model

In [ ]:
# Initilize base model
final_base_model = EfficientNetB0(
    input_shape=(96, 96, 3),
    include_top= False,
    weights='imagenet'
)
final_base_model.trainable = False

# Build classifer
x = final_base_model.output
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output = Dense(2, activation='softmax')(x)
final_full_model = Model(inputs = final_base_model.input, outputs = output)

# Compile model
final_full_model.compile(
    optimizer = OPTIMIZER,
    loss='categorical_crossentropy',
    metrics=['accuracy'],
    )

### Set Trainable layers

In [ ]:
#TRAINABLE_LAYERS = 10
#for layer in final_full_model.layers[0 : TRAINABLE_LAYERS]:
#    layer.trainable = True   

### Train Final Model

In [ ]:

#hist_final_m = final_full_model.fit(train, validation_data = val, epochs = EPOCHS)

### Results

In [ ]:
#plot_hist_sideways(hist_final_m)

In [ ]:
#print(final_full_model.evaluate(test))